My room bookcase
=============

#Liatorp bookcase
http://www.ikea.com/it/it/catalog/products/00116595/
![liatorp](./images/liatorp)


In [3]:
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.006451 seconds


In [4]:
def back(height,width):
    """
    back: builds the back of the bookcase
    example: back()
    :param back: height, width in meters
          thickness is fixed
    :return: a CUBOID. The graphical rappresentation of the back
    """ 
    return CUBOID([width,height,0.1]) #0.1 is thickness

In [5]:
b = back(2.14,8.6)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1816f51690> >

In [111]:
def shelf(width, height, length):
    """
    shelf: builds a shelf
    :param width: width of the shelf
    :return: a CUBOID. The graphical rappresentation of a shelf of fixed measures
    """
    return CUBOID([width,height,length])


def shelves(nshelves, heightBack, width, height, length):
    """
    shelves builds nshelves shelves
    :param nshelves: number of shelves
    :param heightBack: height of the back, the area where shelves are positionable
    :param height: height of the shelf
    :return: nshelves CUBOIDs. nshelves shelves
    """
    interval = heightBack/(nshelves+1)
    s = STRUCT(NN(nshelves) ([shelf(width, height, length), T(2)(interval)]))
    return s

In [122]:
ss = shelves(nshelves=4, heightBack=214, width=90, height=1, length=30)
VIEW(ss)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1819c39930> >

In [124]:
def side(heigth, length):
    return CUBOID([1, heigth, length])

In [127]:
side = side(214,32)
VIEW(side)

TypeError: 'Hpc' object is not callable